In [8]:
%load_ext autoreload
%autoreload 2

#standard imports
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.append("..")

#rl book imports
import rl
from rl.markov_decision_process import MarkovDecisionProcess
from rl.markov_process import State, MarkovProcess, NonTerminal, Terminal

from typing import (Callable, Dict, Generic, Iterator, Iterable, List,
                    Mapping, Optional, Sequence, Tuple, TypeVar, overload)


from rl.distribution import Categorical, Distribution, Constant, Choose
from rl.policy import Policy
from rl.monte_carlo import epsilon_greedy_policy, greedy_policy_from_qvf, glie_mc_control, mc_prediction
from rl.function_approx import LinearFunctionApprox, AdamGradient
from rl.td import glie_sarsa, q_learning
from rl.td_lambda import td_lambda_prediction


#custom imports 
import utils as u
import data as dat
import mdp_agent as ag
import baseline_policies as bp
import q_plots as qp
import backtest as btest
import v_predictor as v_true
import v_convergence as v_conv 
import v_plots2 as v2

from sklearn.linear_model import LinearRegression

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# DATA

In [32]:
constant_params = {
    "mu":   lambda t : 100,
    "sigma": lambda t : 0.2,
    "kappa":  lambda t : 1/10
}

time_varying_params = {
    "mu":   lambda t : 100 + min(t,100)*0.01,
    "kappa":   lambda t : 1/(15+2*np.cos(np.pi*t/30)),
    "sigma":    lambda t: 0.2    
}

params = time_varying_params

In [33]:
train, test = dat.build_simulated_train_test(start="2019-01-01",end="2021-01-01",N=10,mu=params["mu"],sigma=params["sigma"],kappa=params["kappa"])
u.plot_plotly_multiple(train)

# Feature engineering

In [36]:
from sklearn.linear_model import LinearRegression

def ts_features(df_, span=200, trend_window=100,ar1_window=50,last_weight_trend = -0.5, last_weight_ar1 = -1):
    df = df_.copy()
    column_name = df.columns[0]

    sigma_t = df[column_name].ewm(span=span, adjust=False).std()
    mu_t = df[column_name].ewm(span=span, adjust=False).mean()

    # Update the DataFrame with the new computations
    df['mu_t'] = mu_t
    df['sigma_t'] = sigma_t
    df['S_down'] = mu_t - 1.5*sigma_t
    df['S_up'] = mu_t + 1.5*sigma_t
    df["z_score"] = (df[column_name] - mu_t) / sigma_t

    # Compute the trend for mu_t
    def compute_trend(y):
        window_size = len(y)
        weights = np.exp(np.linspace(last_weight_trend, 0, window_size))
        weights /= weights.sum()
        
        lr = LinearRegression()
        X = np.arange(window_size).reshape(-1, 1)
        lr.fit(X, y, sample_weight=weights)
        return lr.coef_[0]

    trend = df['mu_t'].rolling(window=trend_window, min_periods=1).apply(compute_trend, raw=False)
    df['mu_trend'] = trend

    # Compute AR(1) coefficient a
    def compute_ar1(y):
        if len(y) < 2:  # Not enough data to compute AR(1)
            return np.nan
        weights = np.exp(np.linspace(last_weight_ar1, 0, len(y)))
        weights /= weights.sum()

        # Target variable (y) is the series shifted by one, predictor (X) is the original series
        X = y[:-1].reshape(-1, 1)
        y_target = y[1:]
        
        lr = LinearRegression()
        lr.fit(X, y_target, sample_weight=weights[:-1])
        return lr.coef_[0]  # Coefficient a

    ar1_coeff = df[column_name].rolling(window=ar1_window, min_periods=2).apply(
        lambda x: compute_ar1(x.values), raw=False)
    df['ar1_coeff'] = ar1_coeff

    return df

In [37]:
df = ts_features(test)

u.plot_plotly_multiple([df[[df.columns[0]]],df[["mu_t"]],df[["S_down"]],df[["S_up"]]])

u.plot_plotly(df[["mu_trend"]])

u.plot_plotly(df[["z_score"]])

u.plot_plotly(df[["ar1_coeff"]])